In [52]:
import pandas as pd
import numpy as np
import scipy
import os

import matplotlib.pyplot as plt


from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import brier_score_loss, precision_score, recall_score,f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve

from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import KMeans

import random

from scipy.stats import ttest_ind

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
!pip install pyspark

In [55]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Medicare Fraud Detection").getOrCreate()

In [56]:
PartDRawData = "/content/drive/MyDrive/Medicare Part D Prescribers - by Provider and Drug/Medicare Part D Prescribers - by Provider and Drug/2021/MUP_DPR_RY23_P04_V10_DY21_NPIBN.csv"

In [57]:
data = spark.read.csv(PartDRawData, header=True, inferSchema=True)

In [58]:
cols_to_use = ['Prscrbr_NPI', 'Prscrbr_City', 'Prscrbr_State_Abrvtn', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name','Prscrbr_Type', 'Brnd_Name', 'Gnrc_Name', 'Tot_Drug_Cst', 'Tot_Clms', 'Tot_Day_Suply' ]

partDdf = data.select(cols_to_use)

In [59]:
partDdf.show()

+-----------+------------+--------------------+---------------------+------------------+-----------------+--------------------+--------------------+------------+--------+-------------+
|Prscrbr_NPI|Prscrbr_City|Prscrbr_State_Abrvtn|Prscrbr_Last_Org_Name|Prscrbr_First_Name|     Prscrbr_Type|           Brnd_Name|           Gnrc_Name|Tot_Drug_Cst|Tot_Clms|Tot_Day_Suply|
+-----------+------------+--------------------+---------------------+------------------+-----------------+--------------------+--------------------+------------+--------+-------------+
| 1003000126|    Bethesda|                  MD|            Enkeshafi|           Ardalan|Internal Medicine|  Alendronate Sodium|  Alendronate Sodium|      125.28|      11|          930|
| 1003000126|    Bethesda|                  MD|            Enkeshafi|           Ardalan|Internal Medicine| Amlodipine Besylate| Amlodipine Besylate|      812.86|      64|         5311|
| 1003000126|    Bethesda|                  MD|            Enkeshafi|      

In [60]:
cols_to_use = ['Prscrbr_NPI','Gnrc_Name','Tot_Drug_Cst','Tot_Clms','Tot_Day_Suply','Prscrbr_Type']

partDdf1 = data. select(cols_to_use)

column_types = data.dtypes

# Print each column name and its data type
for column, dtype in column_types:
    print(f"Column: {column}, Type: {dtype}")
#partD_Drug_pd['Prscrbr_NPI'] = partD_Drug_pd.Prscrbr_NPI.astype(object)

Column: Prscrbr_NPI, Type: int
Column: Prscrbr_Last_Org_Name, Type: string
Column: Prscrbr_First_Name, Type: string
Column: Prscrbr_City, Type: string
Column: Prscrbr_State_Abrvtn, Type: string
Column: Prscrbr_State_FIPS, Type: string
Column: Prscrbr_Type, Type: string
Column: Prscrbr_Type_Src, Type: string
Column: Brnd_Name, Type: string
Column: Gnrc_Name, Type: string
Column: Tot_Clms, Type: int
Column: Tot_30day_Fills, Type: double
Column: Tot_Day_Suply, Type: int
Column: Tot_Drug_Cst, Type: double
Column: Tot_Benes, Type: int
Column: GE65_Sprsn_Flag, Type: string
Column: GE65_Tot_Clms, Type: int
Column: GE65_Tot_30day_Fills, Type: double
Column: GE65_Tot_Drug_Cst, Type: double
Column: GE65_Tot_Day_Suply, Type: int
Column: GE65_Bene_Sprsn_Flag, Type: string
Column: GE65_Tot_Benes, Type: int


In [61]:
cols_to_use = ['Prscrbr_NPI','Prscrbr_Type']
partDdf2= data.select(cols_to_use)

In [62]:
partDdf2.limit(0).show()

+-----------+------------+
|Prscrbr_NPI|Prscrbr_Type|
+-----------+------------+
+-----------+------------+



In [63]:
partDdf1.limit(0).show()

+-----------+---------+------------+--------+-------------+------------+
|Prscrbr_NPI|Gnrc_Name|Tot_Drug_Cst|Tot_Clms|Tot_Day_Suply|Prscrbr_Type|
+-----------+---------+------------+--------+-------------+------------+
+-----------+---------+------------+--------+-------------+------------+



In [64]:
cols_to_use =['Prscrbr_NPI','Prscrbr_City','Prscrbr_State_Abrvtn', \
                                               'Prscrbr_Last_Org_Name', \
                                               'Prscrbr_First_Name','Prscrbr_Type']
partDdf3 = data.select(cols_to_use)

In [65]:
partDdf3.limit(0).show()

+-----------+------------+--------------------+---------------------+------------------+------------+
|Prscrbr_NPI|Prscrbr_City|Prscrbr_State_Abrvtn|Prscrbr_Last_Org_Name|Prscrbr_First_Name|Prscrbr_Type|
+-----------+------------+--------------------+---------------------+------------------+------------+
+-----------+------------+--------------------+---------------------+------------------+------------+



In [66]:
partDdf3.count()

25231862

In [67]:
partDdf3 = partDdf3.dropDuplicates()

In [68]:
partDdf3.count()

1017417

In [69]:
partDdf3 = partDdf3.withColumnRenamed("Prscrbr_First_Name","first_name")\
                   .withColumnRenamed("Prscrbr_Last_Org_Name","last_name")\
                   .withColumnRenamed("Prscrbr_City","city")\
                   .withColumnRenamed("Prscrbr_State_Abrvtn","state")\
                   .withColumnRenamed("Prscrbr_Type","Speciality")

In [70]:
partDdf3.limit(0).show()

+-----------+----+-----+---------+----------+----------+
|Prscrbr_NPI|city|state|last_name|first_name|Speciality|
+-----------+----+-----+---------+----------+----------+
+-----------+----+-----+---------+----------+----------+



In [73]:
from pyspark.sql import functions as F
group_cols = ['Prscrbr_NPI']
agg_dict = {
    'Tot_Drug_Cst': [F.sum('Tot_Drug_Cst').alias('sum_Tot_Drug_Cst'),
                     F.mean('Tot_Drug_Cst').alias('mean_Tot_Drug_Cst'),
                     F.max('Tot_Drug_Cst').alias('max_Tot_Drug_Cst')],
    'Tot_Clms': [F.sum('Tot_Clms').alias('sum_Tot_Clms'),
                 F.mean('Tot_Clms').alias('mean_Tot_Clms'),
                 F.max('Tot_Clms').alias('max_Tot_Clms')],
    'Tot_Day_Suply': [F.sum('Tot_Day_Suply').alias('sum_Tot_Day_Suply'),
                      F.mean('Tot_Day_Suply').alias('mean_Tot_Day_Suply'),
                      F.max('Tot_Day_Suply').alias('max_Tot_Day_Suply')]
}

# Perform the grouping and aggregation
aggregated_df = data.groupBy(group_cols).agg(*[item for sublist in agg_dict.values() for item in sublist])

# Show the result to verify
#aggregated_df.show()


In [74]:
aggregated_df = aggregated_df.withColumnRenamed("sum_Tot_Drug_Cst", "Total_Drug_Cost_Sum")\
                             .withColumnRenamed("mean_Tot_Drug_Cst", "Total_Drug_Cost_Mean")\
                             .withColumnRenamed("max_Tot_Drug_Cst", "Total_Drug_Cost_Max")\
                             .withColumnRenamed("sum_Tot_Clms", "Total_Claims_Sum")\
                             .withColumnRenamed("mean_Tot_Clms", "Total_Claims_Mean")\
                             .withColumnRenamed("max_Tot_Clms", "Total_Claims_Max")\
                             .withColumnRenamed("sum_Tot_Day_Suply", "Total_Day_Supply_Sum")\
                             .withColumnRenamed("mean_Tot_Day_Suply", "Total_Day_Supply_Mean")\
                             .withColumnRenamed("max_Tot_Day_Suply", "Total_Day_Supply_Max")

# Display the DataFrame to verify the new column names
#aggregated_df.show()

In [ ]:
#aggregated_df.limit(0).show()

In [ ]:
#aggregated_df.count()


In [75]:
partD_all = partDdf3.join(aggregated_df, on='Prscrbr_NPI', how='left')
# Show some of the results to verify the join
#partD_all.show()

In [ ]:
#partD_all.count()

In [76]:
PaymentRawData = "/content/drive/MyDrive/OP_DTL_GNRL_PGYR2022_P01182024.csv"


In [77]:
data = spark.read.csv(PaymentRawData, header=True, inferSchema=True)

In [78]:
cols_to_use = ['Covered_Recipient_NPI', 'Covered_Recipient_First_Name', 'Covered_Recipient_Last_Name', 'Recipient_City', 'Recipient_State', 'Total_Amount_of_Payment_USDollars']
paymentDf = data.select(cols_to_use)

In [79]:
cols_to_use = ['Covered_Recipient_First_Name',\
                  'Covered_Recipient_Last_Name', \
                  'Recipient_City', \
                  'Recipient_State', \
                  'Total_Amount_of_Payment_USDollars']

paymentdf_1 = data.select(cols_to_use)

In [80]:
#paymentdf_1.limit(5).show()

In [81]:
#paymentdf_1.count()

In [82]:
paymentdf_2 = paymentdf_1.groupBy(
    'Covered_Recipient_First_Name',
    'Covered_Recipient_Last_Name',
    'Recipient_City',
    'Recipient_State'
).agg(
    F.sum('Total_Amount_of_Payment_USDollars').alias('Total_Payment_Sum')
).withColumn("Total_Payment_Sum", F.col("Total_Payment_Sum").cast("float"))

# Show the results to verify
#paymentdf_2.show()

In [83]:
rename_dict = {
    'Covered_Recipient_First_Name': 'first_name',
    'Covered_Recipient_Last_Name': 'last_name',
    'Recipient_City': 'city',
    'Recipient_State': 'state',
    'Total_Payment_Sum': 'Total_Payment_Sum'
}

# Apply renaming using withColumnRenamed
for old_name, new_name in rename_dict.items():
    paymentdf_2 = paymentdf_2.withColumnRenamed(old_name, new_name)

# Show the DataFrame to verify changes
#paymentdf_2.show()

In [84]:
from pyspark.sql.functions import col

paymentdf_2 = paymentdf_2.orderBy(col("Total_Payment_Sum").desc())

In [85]:
from pyspark.sql.functions import upper

for column in partD_all.columns:
    partD_all = partD_all.withColumn(column, upper(col(column).cast("string")))


In [86]:
for column in paymentdf_2.columns:
    paymentdf_2 = paymentdf_2.withColumn(column, upper(col(column).cast("string")))

In [87]:
paymentdf_2.limit(1).show()
partD_all.limit(1).show()

+----------+---------+------------+-----+-----------------+
|first_name|last_name|        city|state|Total_Payment_Sum|
+----------+---------+------------+-----+-----------------+
|      NULL|     NULL|PHILADELPHIA|   PA|      7.1777747E8|
+----------+---------+------------+-----+-----------------+

+-----------+----------+-----+---------+------------+--------------+-------------------+--------------------+-------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+
|Prscrbr_NPI|      city|state|last_name|  first_name|    Speciality|Total_Drug_Cost_Sum|Total_Drug_Cost_Mean|Total_Drug_Cost_Max|Total_Claims_Sum|Total_Claims_Mean|Total_Claims_Max|Total_Day_Supply_Sum|Total_Day_Supply_Mean|Total_Day_Supply_Max|
+-----------+----------+-----+---------+------------+--------------+-------------------+--------------------+-------------------+----------------+-----------------+----------------+--------------------+-----

In [90]:
pay_partD_all = partD_all.join(
    paymentdf_2,
    (partD_all.last_name == paymentdf_2.last_name) &
    (partD_all.first_name == paymentdf_2.first_name) &
    (partD_all.city == paymentdf_2.city) &
    (partD_all.state == paymentdf_2.state),
    how='left'
)

In [91]:
pay_partD_all.limit(1).show()

+-----------+----------+-----+---------+------------+--------------+-------------------+--------------------+-------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+------------+---------+----------+-----+-----------------+
|Prscrbr_NPI|      city|state|last_name|  first_name|    Speciality|Total_Drug_Cost_Sum|Total_Drug_Cost_Mean|Total_Drug_Cost_Max|Total_Claims_Sum|Total_Claims_Mean|Total_Claims_Max|Total_Day_Supply_Sum|Total_Day_Supply_Mean|Total_Day_Supply_Max|  first_name|last_name|      city|state|Total_Payment_Sum|
+-----------+----------+-----+---------+------------+--------------+-------------------+--------------------+-------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+------------+---------+----------+-----+-----------------+
| 1003012014|PROVIDENCE|   RI|   SCARFO|KEITH-AUSTIN|ANESTHESIOLOGY| 15256.320000000002|

In [115]:
columns_to_drop = ['last_name', 'first_name', 'city', 'state']
for col_name in columns_to_drop:
    pay_partD_all = pay_partD_all.drop(paymentdf_2[col_name])

# Show some of the results to verify the join and the drop of columns
pay_partD_all.limit(1).show()

+----------+----------+-----+---------+------------+--------------+-------------------+--------------------+-------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+-----------------+
|       NPI|      city|state|last_name|  first_name|    Speciality|Total_Drug_Cost_Sum|Total_Drug_Cost_Mean|Total_Drug_Cost_Max|Total_Claims_Sum|Total_Claims_Mean|Total_Claims_Max|Total_Day_Supply_Sum|Total_Day_Supply_Mean|Total_Day_Supply_Max|Total_Payment_Sum|
+----------+----------+-----+---------+------------+--------------+-------------------+--------------------+-------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+-----------------+
|1003012014|PROVIDENCE|   RI|   SCARFO|KEITH-AUSTIN|ANESTHESIOLOGY| 15256.320000000002|            1525.632|            7156.88|             899|             89.9|             451|               33937|          

In [92]:
pay_partD_all.count()

1269287

In [93]:
IELErawdata = "/content/drive/MyDrive/LEIE.csv"
data = spark.read.csv(IELErawdata, header=True, inferSchema=True)

In [112]:
data.limit(1).show()

+--------+---------+-------+--------------------+--------------+----------+----+---+----+--------------------+--------+-----+-----+--------+--------+--------+----------+--------+
|LASTNAME|FIRSTNAME|MIDNAME|             BUSNAME|       GENERAL| SPECIALTY|UPIN|NPI| DOB|             ADDRESS|    CITY|STATE|  ZIP|EXCLTYPE|EXCLDATE|REINDATE|WAIVERDATE|WVRSTATE|
+--------+---------+-------+--------------------+--------------+----------+----+---+----+--------------------+--------+-----+-----+--------+--------+--------+----------+--------+
|    NULL|     NULL|   NULL|#1 MARKETING SERV...|OTHER BUSINESS|SOBER HOME|NULL|  0|NULL|239 BRIGHTON BEAC...|BROOKLYN|   NY|11235|  1128a1|20200319|       0|         0|    NULL|
+--------+---------+-------+--------------------+--------------+----------+----+---+----+--------------------+--------+-----+-----+--------+--------+--------+----------+--------+



In [96]:
cols_to_use = ['NPI', 'EXCLTYPE']

npifraudDf= data.select(cols_to_use)

In [97]:
npifraudDf.limit(1).show()

+---+--------+
|NPI|EXCLTYPE|
+---+--------+
|  0|  1128a1|
+---+--------+



In [101]:
npifraudDf_1 = npifraudDf.filter("NPI != 0")


In [103]:
npifraudDf_1.count()

7328

In [104]:
npi_fraud_df = npifraudDf_1.withColumnRenamed("NPI", "npi")\
                           .withColumnRenamed("EXCLTYPE", "is_fraud")

# Show the DataFrame schema to verify the new column names
npi_fraud_df.printSchema()

root
 |-- npi: integer (nullable = true)
 |-- is_fraud: string (nullable = true)



In [109]:
from pyspark.sql.functions import lit

npi_fraud_df = npi_fraud_df.withColumn("is_fraud", lit(1))

# Show some of the results to verify the changes
npi_fraud_df.show()

+----------+--------+
|       npi|is_fraud|
+----------+--------+
|1972902351|       1|
|1922348218|       1|
|1942476080|       1|
|1275600959|       1|
|1891731758|       1|
|1265830335|       1|
|1851631543|       1|
|1902198435|       1|
|1073916631|       1|
|1437510278|       1|
|1073682936|       1|
|1902166028|       1|
|1992906937|       1|
|1104947944|       1|
|1164669479|       1|
|1043302250|       1|
|1801231436|       1|
|1912011800|       1|
|1780812768|       1|
|1447560867|       1|
+----------+--------+
only showing top 20 rows



In [116]:
# Features Engineering
pay_partD_all = pay_partD_all.withColumnRenamed("Prscrbr_NPI", "NPI")
Features_df = pay_partD_all.join(
    npi_fraud_df,
    on='npi',  # Join condition
    how='left'  # Type of join
)

In [117]:
Features_df.limit(1).show()

+----------+----------+-----+---------+------------+--------------+-------------------+--------------------+-------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+-----------------+--------+
|       NPI|      city|state|last_name|  first_name|    Speciality|Total_Drug_Cost_Sum|Total_Drug_Cost_Mean|Total_Drug_Cost_Max|Total_Claims_Sum|Total_Claims_Mean|Total_Claims_Max|Total_Day_Supply_Sum|Total_Day_Supply_Mean|Total_Day_Supply_Max|Total_Payment_Sum|is_fraud|
+----------+----------+-----+---------+------------+--------------+-------------------+--------------------+-------------------+----------------+-----------------+----------------+--------------------+---------------------+--------------------+-----------------+--------+
|1003012014|PROVIDENCE|   RI|   SCARFO|KEITH-AUSTIN|ANESTHESIOLOGY| 15256.320000000002|            1525.632|            7156.88|             899|             89.9|             451|    

In [118]:
statistics_df = Features_df.describe()
statistics_df.show()

+-------+--------------------+-----------------+-------+-----------+-------------+--------------------+-------------------+--------------------+-------------------+------------------+-----------------+------------------+--------------------+---------------------+--------------------+------------------+--------+
|summary|                 NPI|             city|  state|  last_name|   first_name|          Speciality|Total_Drug_Cost_Sum|Total_Drug_Cost_Mean|Total_Drug_Cost_Max|  Total_Claims_Sum|Total_Claims_Mean|  Total_Claims_Max|Total_Day_Supply_Sum|Total_Day_Supply_Mean|Total_Day_Supply_Max| Total_Payment_Sum|is_fraud|
+-------+--------------------+-----------------+-------+-----------+-------------+--------------------+-------------------+--------------------+-------------------+------------------+-----------------+------------------+--------------------+---------------------+--------------------+------------------+--------+
|  count|             1269289|          1269289|1269289|    1

In [120]:
# Fill missing values with 0
Features_df = Features_df.fillna(0)

# Show some of the results to verify the fill operation
#Features_df.show()

In [121]:
fraud_count = Features_df.filter(Features_df.is_fraud == 1).count()

# Print the count of fraudulent cases
print(f"Count of 'is_fraud' == 1: {fraud_count}")

Count of 'is_fraud' == 1: 332


In [122]:
FeaturesAll_pd=Features_df

In [127]:
from pyspark.sql.functions import col, log10

# Applying log10 scaling and creating new features using the correct column names
FeaturesAll_pd = FeaturesAll_pd.withColumn("total_drug_cost_sum", log10(col("total_drug_cost_sum") + 1.0))\
                               .withColumn("Total_Claims_Sum", log10(col("Total_Claims_Sum") + 1.0))\
                               .withColumn("total_day_supply_sum", log10(col("total_day_supply_sum") + 1.0))\
                               .withColumn("Total_Payment_Sum", log10(col("Total_Payment_Sum") + 1.0))\
                               .withColumn("total_drug_cost_mean", log10(col("total_drug_cost_mean") + 1.0))\
                               .withColumn("Total_Claims_Mean", log10(col("Total_Claims_Mean") + 1.0))\
                               .withColumn("total_day_supply_mean", log10(col("total_day_supply_mean") + 1.0))\
                               .withColumn("total_drug_cost_max", log10(col("total_drug_cost_max") + 1.0))\
                               .withColumn("Total_Claims_Max", log10(col("Total_Claims_Max") + 1.0))\
                               .withColumn("total_day_supply_max", log10(col("total_day_supply_max") + 1.0))\
                               .withColumn("claim_max-mean", col("Total_Claims_Max") - col("Total_Claims_Mean"))\
                               .withColumn("supply_max-mean", col("total_day_supply_max") - col("total_day_supply_mean"))\
                               .withColumn("drug_max-mean", col("total_drug_cost_max") - col("total_drug_cost_mean"))



In [128]:
FeaturesAll_pd

DataFrame[NPI: string, city: string, state: string, last_name: string, first_name: string, Speciality: string, total_drug_cost_sum: double, total_drug_cost_mean: double, total_drug_cost_max: double, Total_Claims_Sum: double, Total_Claims_Mean: double, Total_Claims_Max: double, total_day_supply_sum: double, total_day_supply_mean: double, total_day_supply_max: double, Total_Payment_Sum: double, is_fraud: int, claim_max-mean: double, supply_max-mean: double, drug_max-mean: double]

In [ ]:
#FeaturesAll_pd['npi'] = FeaturesAll_pd.npi.astype(object)

In [129]:
categorical_features = ['npi','last_name', 'Speciality','first_name','city', 'state']

In [136]:
numerical_features = [
    'total_drug_cost_sum', 'total_drug_cost_mean', 'Total_Payment_Sum',
    'total_drug_cost_max', 'Total_Claims_Sum', 'Total_Claims_Mean', 'Total_Claims_Max',
    'total_day_supply_sum', 'total_day_supply_mean', 'total_day_supply_max',
    'claim_max-mean', 'supply_max-mean', 'drug_max-mean'
]

In [137]:
target = ['is_fraud']

In [138]:
allvars = categorical_features + numerical_features + target

In [140]:
# Assuming 'FeaturesAll_df' is your DataFrame and 'numerical_features' lists all feature columns
from pyspark.ml.feature import VectorAssembler

# First, ensure the 'numerical_features' list does not contain the target variable
feature_vars = [col for col in numerical_features if col != 'is_fraud']

# Use VectorAssembler to combine feature columns into a single feature vector
assembler = VectorAssembler(inputCols=feature_vars, outputCol="features")
x = assembler.transform(FeaturesAll_pd)

# Select the target variable and the features vector
y = features_df.select("features", "is_fraud")

In [ ]:
# scikit learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_valid.shape)

In [ ]:
X_train[numerical_features] = X_train.loc[:,numerical_features].fillna(0)
X_valid[numerical_features] = X_valid.loc[:,numerical_features].fillna(0)
X_train[categorical_features] = X_train.loc[:,categorical_features].fillna('NA')
X_valid[categorical_features] = X_valid.loc[:,categorical_features].fillna('NA')

In [ ]:
scaler= StandardScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features].values)
X_valid[numerical_features] = scaler.transform(X_valid[numerical_features].values)

In [ ]:
print(X_train[numerical_features].dtypes)

In [ ]:
ix_ran = FeaturesAll_pd.index.values
np.random.shuffle(ix_ran)

df_len = len(FeaturesAll_pd)
train_len = int(df_len * 0.8)  # 80% for training


ix_train = ix_ran[:train_len]
ix_valid = ix_ran[train_len:]

df_train = FeaturesAll_pd.ix[ix_train]
df_valid = FeaturesAll_pd.ix[ix_valid]

print(len(ix_train))
print(len(ix_valid))

In [ ]:
print(df_train.dtypes)

In [ ]:
# Drug Weighted_Scores

partD_drug_train = pd.merge(partD_Drug_pd,df_train[['npi','is_fraud']], how='inner', on=['npi'])
partD_drug_All = pd.merge(partD_Drug_pd,FeaturesAll_pd[['npi','is_fraud']], how='inner', on=['npi'])

In [ ]:
print(len(partD_drug_train[partD_drug_train['is_fraud']==1]))

In [ ]:
# get unique drug names
drugs = set([ drugx for drugx in partD_drug_train['drug_name'].values if isinstance(drugx, str)])
print(len(drugs))

In [ ]:
print("Total records in train set : ")
print(len(partD_drug_train))
print("Total Fraud in train set : ")
print(len(partD_drug_train[partD_drug_train['is_fraud']==1]))
partD_drug_train.head()

In [ ]:
cols = ['total_drug_cost','total_claim_count','total_day_supply']

In [ ]:
partD_drug_train_Group = partD_drug_train.groupby(['drug_name', 'is_fraud'])
partD_drug_All_Group = partD_drug_All.groupby(['drug_name', 'is_fraud'])

In [ ]:
drug_keys = partD_drug_train_Group.groups.keys()
print(len(drug_keys))

In [ ]:
drug_keys

In [ ]:
drug_with_isfraud = [drugx for drugx in drugs if ((drugx,0.0) in drug_keys ) & ( (drugx,1.0) in drug_keys)]

In [ ]:
from scipy.stats import ttest_ind
re_drug_tt = dict()
for drugx in drug_with_isfraud:
    for colx in cols:
        fraud_0 = partD_drug_train_Group.get_group((drugx,0.0))[colx].values
        fraud_1 = partD_drug_train_Group.get_group((drugx,1.0))[colx].values
        # print len(fraud_0), len(fraud_1)
        if (len(fraud_0)>2) & (len(fraud_1)>2) :
            tt = ttest_ind(fraud_0, fraud_1)
            re_drug_tt[(drugx, colx)] = tt

In [ ]:
#Setting Probilities
Prob_005 = [(key, p) for (key, (t, p)) in re_drug_tt.items() if p <=0.05]
print(len(Prob_005))

In [ ]:
inx=100
drug_name = Prob_005[inx][0][0]
print(drug_name)
df_bar = pd.concat([partD_drug_All_Group.get_group((Prob_005[inx][0][0],0.0)), partD_drug_All_Group.get_group((Prob_005[inx][0][0],1.0))])
df_bar.head()

In [ ]:
Feture_DrugWeighted = []
new_col_all =[]
for i, p005x in enumerate(Prob_005):
    #if i>4:
    #   break
    drug_name = p005x[0][0]
    cat_name = p005x[0][1]

    new_col = drug_name+'_'+cat_name
    new_col_all.append(new_col)

    drug_0 = partD_drug_All_Group.get_group((drug_name,0.0))[['npi', cat_name]]
    drug_1 = partD_drug_All_Group.get_group((drug_name,1.0))[['npi', cat_name]]

    drug_01 = pd.concat([drug_0, drug_1])
    drug_01.rename(columns={cat_name: new_col}, inplace=True)
    Feture_DrugWeighted.append(drug_01)

In [ ]:
npi_col = FeaturesAll_pd[['npi']]

w_npi = []

for n, nx in enumerate(Feture_DrugWeighted):
      nggx = pd.merge(npi_col, nx.drop_duplicates(['npi']), on='npi', how='left')
      w_npi.append(nggx)

In [ ]:
FeaturesAll_pd1 = FeaturesAll_pd

In [ ]:
for wx in w_npi:
    col_n = wx.columns[1]
    FeaturesAll_pd1[col_n] = wx[col_n].values

wx = w_npi[0]
wx.columns[1]
col_n = wx.columns[1]

In [ ]:
len(wx[col_n].values)
FeaturesAll_pd1.fillna(0)

In [ ]:
new_col_all

In [ ]:
FeaturesAll_pd1[new_col_all].describe()

In [ ]:
FeaturesAll_pd1['drug_mean'] = FeaturesAll_pd1[new_col_all].mean(axis=1)


In [ ]:
FeaturesAll_pd['drug_mean'] = FeaturesAll_pd['drug_mean'].map(lambda x: np.log10(x + 1.0))

In [ ]:
FeaturesAll_pd1['drug_sum'] = FeaturesAll_pd1[new_col_all].sum(axis=1)
FeaturesAll_pd['drug_sum'] = FeaturesAll_pd['drug_sum'].map(lambda x: np.log10(x + 1.0))

In [ ]:
FeaturesAll_pd1['drug_variance'] = FeaturesAll_pd1[new_col_all].var(axis=1)

In [ ]:
FeaturesAll_pd1

In [ ]:
df_train = FeaturesAll_pd1.ix[ix_train]
df_valid = FeaturesAll_pd1.ix[ix_valid]

df_train.fillna(0)
df_valid.fillna(0)

In [ ]:
df_valid.columns

In [ ]:
#Create the Specialty Weight
spec_dict =[]
spec_fraud_1 = df_train[df_train['is_fraud']==1]['Speciality']

In [ ]:
from collections import Counter
counts = Counter(spec_fraud_1)
spec_dict =  dict(counts)

In [ ]:

FeaturesAll_pd1['Spec_Weight'] = FeaturesAll_pd1['Speciality'].map(lambda x: spec_dict.get(x, 0))

In [ ]:
df_train = FeaturesAll_pd1.ix[ix_train]
df_valid = FeaturesAll_pd1.ix[ix_valid]

In [ ]:
len(df_train[df_train['is_fraud'] == 1])

In [ ]:
print(df_train.dtypes)

In [ ]:
df_train.fillna(0)

In [ ]:
df_valid.fillna(0)

In [ ]:
numerical_features1 = numerical_features + ['drug_sum','Spec_Weight']

In [ ]:
numerical_features1

In [ ]:
positives=len(df_train[df_train['is_fraud'] == 1])
positives

In [ ]:
dataset_size=len(df_train)
dataset_size

In [ ]:
per_ones=(float(positives)/float(dataset_size))*100
per_ones

In [ ]:
negatives=float(dataset_size-positives)
t=negatives/positives
t


In [ ]:
BalancingRatio= positives/dataset_size
BalancingRatio

In [ ]:
BalancingRatio= positives/dataset_size
BalancingRatio

In [ ]:
X= df_train[numerical_features1].values
Y = df_train['is_fraud'].values
clf =  LogisticRegression(C=1e5, class_weight={0:1, 1:4000}, n_jobs=3)
clf.fit(X,Y)
y_p=clf.predict_proba(X)

In [ ]:
params_0 = {'n_estimators': 100, 'max_depth': 8, 'min_samples_split': 3, 'learning_rate': 0.01}
params_1 = {'n_estimators': 500, 'max_depth': 10, 'min_samples_split': 5, 'class_weight' : {0:1, 1:2514}, 'n_jobs':5}

scaler = StandardScaler()

clfs = [
    LogisticRegression(C=1e5,class_weight= {0:1, 1:2514}, n_jobs=5),

    GaussianNB(),

    ensemble.RandomForestClassifier(**params_1),

    ensemble.ExtraTreesClassifier(**params_1),

    ensemble.GradientBoostingClassifier(**params_0)

    ]

In [ ]:
X_train = df_train[numerical_features1].values

y_train = df_train['is_fraud'].values

X_train = scaler.fit_transform(X_train)

X_valid = df_valid[numerical_features1].values
y_valid = df_valid['is_fraud'].values
X_valid_x= scaler.transform(X_valid)

In [ ]:
prob_result = []
df_m = []
clfs_fited = []
for clf in clfs:
    print("%s:" %  clf.__class__.__name__)
    clf.fit(X_train,y_train)
    clfs_fited.append(clf)
    y_pred = clf.predict(X_valid_x)
    prob_pos  = clf.predict_proba(X_valid_x)[:, 1]
    prob_result.append(prob_pos)
    m = confusion_matrix(y_valid, y_pred)
    clf_score = brier_score_loss(y_valid, prob_pos, pos_label=y_valid.max())
    print("\tBrier: %1.5f" % (clf_score))
    print("\tPrecision: %1.5f" % precision_score(y_valid, y_pred))
    print("\tRecall: %1.5f" % recall_score(y_valid, y_pred))
    print("\tF1: %1.5f" % f1_score(y_valid, y_pred))
    print("\tauc: %1.5f" % roc_auc_score(y_valid, prob_pos))
    print("\tAccuracy: %1.5f\n" % accuracy_score(y_valid, y_pred))
    df_m.append(
        pd.DataFrame(m, index=['True Negative', 'True Positive'], columns=['Pred. Negative', 'Pred. Positive'])
        )

In [ ]:
fpr, tpr, thresholds = roc_curve(y_valid, prob_result[2])

In [ ]:
fpr, tpr, thresholds = roc_curve(y_valid, prob_result[2])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=1, label='ROC (area = %0.2f)' % roc_auc)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
m

In [ ]:
X_valid_x[0]

In [ ]:
df_train

In [ ]:
X_valid_x[1]

In [ ]:
y_pred = clf.predict(X_valid_x)

In [ ]:
y_pred

In [ ]:
X_train[0]

In [ ]:
feature_importance = clfs_fited[2].feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)

In [ ]:
feature_importance[sorted_idx]


In [ ]:
features = [numerical_features1[ix] for ix in sorted_idx]
bardata = {"name":features[::-1], "importance percent":feature_importance[sorted_idx][::-1]}

In [ ]:
plt.figure()

# Create plot title
plt.title("Feature Importance")

# Add bars
plt.bar(range(X.shape[1]), feature_importance[sorted_idx])

# Add feature names as x-axis labels
plt.xticks(range(X.shape[1]), features, rotation=90)

# Show plot
plt.show()